In [1]:
#implement a perceptron

import os
import numpy as np
from PIL import Image

class Perceptron:
    def __init__(self, num_features):
        self.weights = np.zeros(num_features + 1)  # +1 for the bias
        self.learning_rate = 0.1
    
    def predict(self, features):
        # Add bias term (x0 = 1)
        activation = np.dot(features, self.weights[1:]) + self.weights[0]
        return 1 if activation >= 0 else 0
    
    def train(self, X_train, y_train, X_val, y_val, epochs):
        best_val_accuracy = 0
        best_weights = np.copy(self.weights)
        
        for epoch in range(epochs):
            for features, label in zip(X_train, y_train):
                prediction = self.predict(features)
                error = label - prediction
                self.weights[1:] += self.learning_rate * error * features
                self.weights[0] += self.learning_rate * error
            
            # Evaluate on validation set
            val_accuracy = self.evaluate(X_val, y_val)
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                best_weights = np.copy(self.weights)
        
        # Set the best weights found during training
        self.weights = best_weights
    
    def evaluate(self, X, y):
        correct = 0
        for features, label in zip(X, y):
            prediction = self.predict(features)
            if prediction == label:
                correct += 1
        accuracy = correct / len(y)
        return accuracy

def load_and_prepare_data(folder_path):
    files = os.listdir(folder_path)
    X = []
    y = []
    
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        
        try:
            # Load image and convert to grayscale and then to numpy array
            img = Image.open(file_path).convert('L')  # Convert to grayscale
            img = img.resize((64, 64))  # Resize image if needed
            arr = np.array(img)
            
            # Flatten array and normalize
            features = arr.flatten() / 255.0  # Normalize pixel values
            
            # Assuming file names are labeled as 'class_name.xxx.jpg'
            label = 1 if 'class_name' in file_name else 0
            
            X.append(features)
            y.append(label)
        
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            continue  # Skip this file and continue with the next one
    
    return np.array(X), np.array(y)

# Example usage:
train_folder_path = '/home/rekha-habshipuram/Downloads/archive/train_data'
val_folder_path = '/home/rekha-habshipuram/Downloads/archive/val_data'
test_folder_path = '/home/rekha-habshipuram/Downloads/archive/test_data'

# Load and prepare training, validation, and testing data
X_train, y_train = load_and_prepare_data(train_folder_path)
X_val, y_val = load_and_prepare_data(val_folder_path)
X_test, y_test = load_and_prepare_data(test_folder_path)

# Shuffle the training data
shuffle_index = np.random.permutation(len(X_train))
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# Initialize and train the perceptron
perceptron = Perceptron(num_features=X_train.shape[1])
perceptron.train(X_train, y_train, X_val, y_val, epochs=10)

# Evaluate the perceptron on training, validation, and testing data
train_accuracy = perceptron.evaluate(X_train, y_train)
val_accuracy = perceptron.evaluate(X_val, y_val)
test_accuracy = perceptron.evaluate(X_test, y_test)

print(f"Training Accuracy: {train_accuracy:.0%}")
print(f"Validation Accuracy: {val_accuracy:.0%}")
print(f"Testing Accuracy: {test_accuracy:.0%}")


Error processing /home/rekha-habshipuram/Downloads/archive/train_data/MNIST: [Errno 21] Is a directory: '/home/rekha-habshipuram/Downloads/archive/train_data/MNIST'
Error processing /home/rekha-habshipuram/Downloads/archive/val_data/MNIST: [Errno 21] Is a directory: '/home/rekha-habshipuram/Downloads/archive/val_data/MNIST'
Error processing /home/rekha-habshipuram/Downloads/archive/test_data/MNIST: [Errno 21] Is a directory: '/home/rekha-habshipuram/Downloads/archive/test_data/MNIST'
Training Accuracy: 100%
Validation Accuracy: 100%
Testing Accuracy: 100%


In [2]:
#Build and train a simple neural network using a framework like TensorFlow or PyTorch

import os
import numpy as np
import tensorflow as tf
from PIL import Image

class SimpleNeuralNetwork:
    def __init__(self, input_shape):
        self.model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=input_shape),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        self.model.compile(optimizer='adam',
                           loss='binary_crossentropy',
                           metrics=['accuracy'])
    
    def train(self, X_train, y_train, X_val, y_val, epochs):
        history = self.model.fit(X_train, y_train,
                                 epochs=epochs,
                                 validation_data=(X_val, y_val),
                                 verbose=1)
        return history
    
    def evaluate(self, X, y):
        _, accuracy = self.model.evaluate(X, y, verbose=0)
        return accuracy
    
    def predict(self, X):
        return (self.model.predict(X) > 0.5).astype("int32")

def load_and_prepare_data(folder_path):
    X = []
    y = []
    
    try:
        files = os.listdir(folder_path)
        
        for file_name in files:
            file_path = os.path.join(folder_path, file_name)

            # Check if the file is a regular file (not a directory)
            if os.path.isfile(file_path):
                try:
                    # Load image and convert to grayscale and then to numpy array
                    img = Image.open(file_path).convert('L')  # Convert to grayscale
                    img = img.resize((64, 64))  # Resize image if needed
                    arr = np.array(img)

                    # Flatten array and normalize
                    features = arr.flatten() / 255.0  # Normalize pixel values

                    # Assuming file names are labeled as 'class_name.xxx.jpg'
                    label = 1 if 'class_name' in file_name else 0

                    X.append(features)
                    y.append(label)

                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
                    continue  # Skip this file and continue with the next one
            else:
                print(f"{file_path} is not a regular file. Skipping.")

    except FileNotFoundError:
        print(f"Directory '{folder_path}' not found.")
    
    return np.array(X), np.array(y)

# Example usage:
train_folder_path = '/home/rekha-habshipuram/Downloads/archive/train_data'
val_folder_path = '/home/rekha-habshipuram/Downloads/archive/val_data'
test_folder_path = '/home/rekha-habshipuram/Downloads/archive/test_data'

# Load and prepare training, validation, and testing data
X_train, y_train = load_and_prepare_data(train_folder_path)
X_val, y_val = load_and_prepare_data(val_folder_path)
X_test, y_test = load_and_prepare_data(test_folder_path)

# Shuffle the training data
shuffle_index = np.random.permutation(len(X_train))
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# Initialize and train the neural network
input_shape = X_train.shape[1:]
model = SimpleNeuralNetwork(input_shape)
history = model.train(X_train, y_train, X_val, y_val, epochs=10)

# Evaluate the model on training, validation, and testing data
train_accuracy = model.evaluate(X_train, y_train)
val_accuracy = model.evaluate(X_val, y_val)
test_accuracy = model.evaluate(X_test, y_test)

# Print accuracies as integers
print(f"Training Accuracy: {train_accuracy * 100:.0f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.0f}%")
print(f"Testing Accuracy: {test_accuracy * 100:.0f}%")

# Individual class accuracies
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

train_class_accuracy = np.mean(y_train == y_train_pred.flatten())
val_class_accuracy = np.mean(y_val == y_val_pred.flatten())
test_class_accuracy = np.mean(y_test == y_test_pred.flatten())

print(f"Training Class Accuracy: {train_class_accuracy * 100:.0f}%")
print(f"Validation Class Accuracy: {val_class_accuracy * 100:.0f}%")
print(f"Testing Class Accuracy: {test_class_accuracy * 100:.0f}%")


2024-08-16 18:10:57.699824: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 18:10:58.011859: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 18:10:58.013979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 18:10:58.454060: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-16 18:11:00.338893: W tensorflow/compiler/tf

/home/rekha-habshipuram/Downloads/archive/train_data/MNIST is not a regular file. Skipping.
/home/rekha-habshipuram/Downloads/archive/val_data/MNIST is not a regular file. Skipping.
/home/rekha-habshipuram/Downloads/archive/test_data/MNIST is not a regular file. Skipping.
Epoch 1/10


/home/rekha-habshipuram/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7963 - loss: 0.2145 - val_accuracy: 1.0000 - val_loss: 2.0849e-07
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 2.4205e-08 - val_accuracy: 1.0000 - val_loss: 1.4934e-08
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 1.2631e-09 - val_accuracy: 1.0000 - val_loss: 6.8307e-09
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 8.3789e-10 - val_accuracy: 1.0000 - val_loss: 5.4218e-09
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 4.4468e-10 - val_accuracy: 1.0000 - val_loss: 5.0726e-09
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 7.1931e-10 - val_accuracy: 1.0000 - val_loss: 4.9781e-09
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 8.8397e-10 - val_accuracy: 1.0000 - val_loss: 4.9519e-09
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss:

In [3]:
#Experiment with different regularization techniques on a neural network

import os
import numpy as np
from PIL import Image

class Perceptron:
    def __init__(self, num_features, regularization=None, lambda_val=0.01):
        self.weights = np.zeros(num_features + 1)  # +1 for the bias
        self.learning_rate = 0.1
        self.regularization = regularization
        self.lambda_val = lambda_val  # Regularization parameter

    def predict(self, features):
        activation = np.dot(features, self.weights[1:]) + self.weights[0]
        return 1 if activation >= 0 else 0

    def train(self, X_train, y_train, X_val, y_val, epochs):
        best_val_accuracy = 0
        best_weights = np.copy(self.weights)

        for epoch in range(epochs):
            for features, label in zip(X_train, y_train):
                prediction = self.predict(features)
                error = label - prediction
                self.weights[1:] += self.learning_rate * error * features
                self.weights[0] += self.learning_rate * error

                if self.regularization == 'l1':
                    self.weights[1:] -= self.lambda_val * np.sign(self.weights[1:])
                elif self.regularization == 'l2':
                    self.weights[1:] -= self.lambda_val * self.weights[1:]  # L2 penalty

            val_accuracy = self.evaluate(X_val, y_val)
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                best_weights = np.copy(self.weights)

        self.weights = best_weights

    def evaluate(self, X, y):
        correct = 0
        for features, label in zip(X, y):
            prediction = self.predict(features)
            if prediction == label:
                correct += 1
        accuracy = correct / len(y)
        return accuracy

def load_and_prepare_data(folder_path):
    X = []
    y = []
    
    try:
        files = os.listdir(folder_path)
        
        for file_name in files:
            file_path = os.path.join(folder_path, file_name)

            # Check if the file is a regular file (not a directory)
            if os.path.isfile(file_path):
                try:
                    # Load image and convert to grayscale and then to numpy array
                    img = Image.open(file_path).convert('L')  # Convert to grayscale
                    img = img.resize((64, 64))  # Resize image if needed
                    arr = np.array(img)

                    # Flatten array and normalize
                    features = arr.flatten() / 255.0  # Normalize pixel values

                    # Assuming file names are labeled as 'class_name.xxx.jpg'
                    label = 1 if 'class_name' in file_name else 0

                    X.append(features)
                    y.append(label)

                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
                    continue  # Skip this file and continue with the next one
            else:
                print(f"{file_path} is not a regular file. Skipping.")

    except FileNotFoundError:
        print(f"Directory '{folder_path}' not found.")
    
    return np.array(X), np.array(y)

# Example usage:
train_folder_path = '/home/rekha-habshipuram/Downloads/archive/train_data'
val_folder_path = '/home/rekha-habshipuram/Downloads/archive/val_data'
test_folder_path = '/home/rekha-habshipuram/Downloads/archive/test_data'

# Load and prepare training, validation, and testing data
X_train, y_train = load_and_prepare_data(train_folder_path)
X_val, y_val = load_and_prepare_data(val_folder_path)
X_test, y_test = load_and_prepare_data(test_folder_path)

# Shuffle the training data
shuffle_index = np.random.permutation(len(X_train))
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# Initialize and train the perceptron with L1 regularization
perceptron_l1 = Perceptron(num_features=X_train.shape[1], regularization='l1', lambda_val=0.01)
perceptron_l1.train(X_train, y_train, X_val, y_val, epochs=10)

# Initialize and train the perceptron with L2 regularization
perceptron_l2 = Perceptron(num_features=X_train.shape[1], regularization='l2', lambda_val=0.01)
perceptron_l2.train(X_train, y_train, X_val, y_val, epochs=10)

# Evaluate the perceptrons on training, validation, and testing data
train_accuracy_l1 = perceptron_l1.evaluate(X_train, y_train)
val_accuracy_l1 = perceptron_l1.evaluate(X_val, y_val)
test_accuracy_l1 = perceptron_l1.evaluate(X_test, y_test)

train_accuracy_l2 = perceptron_l2.evaluate(X_train, y_train)
val_accuracy_l2 = perceptron_l2.evaluate(X_val, y_val)
test_accuracy_l2 = perceptron_l2.evaluate(X_test, y_test)

# Print accuracies as integers
print("L1 Regularization:")
print(f"Training Accuracy: {train_accuracy_l1 * 100:.0f}%")
print(f"Validation Accuracy: {val_accuracy_l1 * 100:.0f}%")
print(f"Testing Accuracy: {test_accuracy_l1 * 100:.0f}%")

print("\nL2 Regularization:")
print(f"Training Accuracy: {train_accuracy_l2 * 100:.0f}%")
print(f"Validation Accuracy: {val_accuracy_l2 * 100:.0f}%")
print(f"Testing Accuracy: {test_accuracy_l2 * 100:.0f}%")


/home/rekha-habshipuram/Downloads/archive/train_data/MNIST is not a regular file. Skipping.
/home/rekha-habshipuram/Downloads/archive/val_data/MNIST is not a regular file. Skipping.
/home/rekha-habshipuram/Downloads/archive/test_data/MNIST is not a regular file. Skipping.
L1 Regularization:
Training Accuracy: 100%
Validation Accuracy: 100%
Testing Accuracy: 100%

L2 Regularization:
Training Accuracy: 100%
Validation Accuracy: 100%
Testing Accuracy: 100%


In [4]:
#Compare performance with various optimization algorithms

import os
import numpy as np
from PIL import Image

class Perceptron:
    def __init__(self, num_features, optimizer='gradient_descent', learning_rate=0.1, batch_size=None):
        self.weights = np.zeros(num_features + 1)  # +1 for the bias
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.batch_size = batch_size
    
    def predict(self, features):
        # Add bias term (x0 = 1)
        activation = np.dot(features, self.weights[1:]) + self.weights[0]
        return np.where(activation >= 0, 1, 0)
    
    def train(self, X_train, y_train, X_val, y_val, epochs):
        best_val_accuracy = 0
        best_weights = np.copy(self.weights)
        
        for epoch in range(epochs):
            if self.optimizer == 'gradient_descent':
                self.gradient_descent(X_train, y_train)
            elif self.optimizer == 'sgd':
                self.stochastic_gradient_descent(X_train, y_train)
            elif self.optimizer == 'mini_batch':
                self.mini_batch_gradient_descent(X_train, y_train)
            
            # Evaluate on validation set
            val_accuracy = self.evaluate(X_val, y_val)
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                best_weights = np.copy(self.weights)
        
        # Set the best weights found during training
        self.weights = best_weights
    
    def gradient_descent(self, X_train, y_train):
        errors = y_train - self.predict(X_train)
        self.weights[1:] += self.learning_rate * np.dot(errors.T, X_train)
        self.weights[0] += self.learning_rate * np.sum(errors)
    
    def stochastic_gradient_descent(self, X_train, y_train):
        for features, label in zip(X_train, y_train):
            prediction = self.predict(features)
            error = label - prediction
            self.weights[1:] += self.learning_rate * error * features
            self.weights[0] += self.learning_rate * error
    
    def mini_batch_gradient_descent(self, X_train, y_train):
        if self.batch_size is None:
            self.batch_size = len(X_train)
        
        for i in range(0, len(X_train), self.batch_size):
            X_batch = X_train[i:i+self.batch_size]
            y_batch = y_train[i:i+self.batch_size]
            
            errors = y_batch - self.predict(X_batch)
            self.weights[1:] += self.learning_rate * np.dot(errors.T, X_batch)
            self.weights[0] += self.learning_rate * np.sum(errors)
    
    def evaluate(self, X, y):
        correct = 0
        for features, label in zip(X, y):
            prediction = self.predict(features)
            if prediction == label:
                correct += 1
        accuracy = correct / len(y)
        return accuracy

def load_and_prepare_data(folder_path):
    files = os.listdir(folder_path)
    X = []
    y = []
    
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.isfile(file_path):  # Check if the path is a file
            # Load image and convert to grayscale and then to numpy array
            img = Image.open(file_path).convert('L')  # Convert to grayscale
            img = img.resize((64, 64))  # Resize image if needed
            arr = np.array(img)
            
            # Flatten array and normalize
            features = arr.flatten() / 255.0  # Normalize pixel values
            
            # Assuming file names are labeled as 'class_name.xxx.jpg'
            label = 1 if 'class_name' in file_name else 0
            
            X.append(features)
            y.append(label)
    
    return np.array(X), np.array(y)

# Example usage:
train_folder_path = '/home/rekha-habshipuram/Downloads/archive/train_data'
val_folder_path = '/home/rekha-habshipuram/Downloads/archive/val_data'
test_folder_path = '/home/rekha-habshipuram/Downloads/archive/test_data'

# Load and prepare training, validation, and testing data
X_train, y_train = load_and_prepare_data(train_folder_path)
X_val, y_val = load_and_prepare_data(val_folder_path)
X_test, y_test = load_and_prepare_data(test_folder_path)

# Shuffle the training data
shuffle_index = np.random.permutation(len(X_train))
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

# Initialize and train the perceptrons with different optimizers
perceptron_gd = Perceptron(num_features=X_train.shape[1], optimizer='gradient_descent', learning_rate=0.1)
perceptron_gd.train(X_train, y_train, X_val, y_val, epochs=10)

perceptron_sgd = Perceptron(num_features=X_train.shape[1], optimizer='sgd', learning_rate=0.1)
perceptron_sgd.train(X_train, y_train, X_val, y_val, epochs=10)

perceptron_mini_batch = Perceptron(num_features=X_train.shape[1], optimizer='mini_batch', learning_rate=0.1, batch_size=32)
perceptron_mini_batch.train(X_train, y_train, X_val, y_val, epochs=10)

# Evaluate the perceptrons on training, validation, and testing data
train_accuracy_gd = perceptron_gd.evaluate(X_train, y_train)
val_accuracy_gd = perceptron_gd.evaluate(X_val, y_val)
test_accuracy_gd = perceptron_gd.evaluate(X_test, y_test)

train_accuracy_sgd = perceptron_sgd.evaluate(X_train, y_train)
val_accuracy_sgd = perceptron_sgd.evaluate(X_val, y_val)
test_accuracy_sgd = perceptron_sgd.evaluate(X_test, y_test)

train_accuracy_mini_batch = perceptron_mini_batch.evaluate(X_train, y_train)
val_accuracy_mini_batch = perceptron_mini_batch.evaluate(X_val, y_val)
test_accuracy_mini_batch = perceptron_mini_batch.evaluate(X_test, y_test)

# Print results
print("Performance with Gradient Descent:")
print(f"Training Accuracy: {train_accuracy_gd:.0%}")
print(f"Validation Accuracy: {val_accuracy_gd:.0%}")
print(f"Testing Accuracy: {test_accuracy_gd:.0%}")

print("\nPerformance with Stochastic Gradient Descent:")
print(f"Training Accuracy: {train_accuracy_sgd:.0%}")
print(f"Validation Accuracy: {val_accuracy_sgd:.0%}")
print(f"Testing Accuracy: {test_accuracy_sgd:.0%}")

print("\nPerformance with Mini-batch Gradient Descent (batch size = 42):")
print(f"Training Accuracy: {train_accuracy_mini_batch:.0%}")
print(f"Validation Accuracy: {val_accuracy_mini_batch:.0%}")
print(f"Testing Accuracy: {test_accuracy_mini_batch:.0%}")


Performance with Gradient Descent:
Training Accuracy: 100%
Validation Accuracy: 100%
Testing Accuracy: 100%

Performance with Stochastic Gradient Descent:
Training Accuracy: 100%
Validation Accuracy: 100%
Testing Accuracy: 100%

Performance with Mini-batch Gradient Descent (batch size = 42):
Training Accuracy: 100%
Validation Accuracy: 100%
Testing Accuracy: 100%


In [6]:
#Create a multi-layer perceptron (MLP) for digit classification (MNIST dataset)

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define transforms for preprocessing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load data from folders
train_data = datasets.MNIST('/home/rekha-habshipuram/Downloads/archive/train_data', download=True, train=True, transform=transform)
validation_data = datasets.MNIST('/home/rekha-habshipuram/Downloads/archive/test_dat', download=True, train=False, transform=transform)
test_data = datasets.MNIST('/home/rekha-habshipuram/Downloads/archive/val_data', download=True, train=False, transform=transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

# Multi-layer perceptron (MLP) model
mlp = nn.Sequential(
    nn.Linear(784, 128),  # input layer (28x28) -> hidden layer (128)
    nn.ReLU(),
    nn.Linear(128, 64),  # hidden layer (128) -> hidden layer (64)
    nn.ReLU(),
    nn.Linear(64, 10)  # hidden layer (64) -> output layer (10)
)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.01, momentum=0.9)

# Train the network
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        optimizer.zero_grad()
        outputs = mlp(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/i}')

# Evaluate on training set
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Training Accuracy: {100 * correct / total} %')

# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Test Accuracy: {100 * correct / total} %')

# Evaluate on validation set
correct = 0
total = 0
with torch.no_grad():
    for data in validation_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Validation Accuracy: {100 * correct / total} %')

Epoch 1, Loss: 0.4266958488757735
Epoch 2, Loss: 0.17506469418210244
Epoch 3, Loss: 0.13070789829946125
Epoch 4, Loss: 0.10558524214649691
Epoch 5, Loss: 0.08751164211865749
Epoch 6, Loss: 0.07552732143271813
Epoch 7, Loss: 0.06824164294285885
Epoch 8, Loss: 0.05799117223150917
Epoch 9, Loss: 0.054232563490864076
Epoch 10, Loss: 0.0461434526472097
Training Accuracy: 98.91833333333334 %
Test Accuracy: 97.67 %
Validation Accuracy: 97.67 %
